## mixtral-200-part* have issues with links
## mixtral-200-<<URL>> are have better links

In [34]:
import os
import json
import random
import string
from pathlib import Path

from datasets import concatenate_datasets, Dataset
from dotenv import load_dotenv

load_dotenv("../../.env")

data_dir = Path(os.environ["PROJECT_HOME_DIR"]) / "data" / "essays"

files = list(data_dir.glob("mixtral-200-<<*.pq"))


# ds = concatenate_datasets([Dataset.from_parquet(str(f)) for f in files])

ds = Dataset.from_parquet("/drive2/kaggle/pii-dd/data/essays/mixtral-v1a.pq")

ds

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['id', 'essay', 'prompt', 'bio', 'guidelines', 'names', 'famous_person'],
    num_rows: 2913
})

In [2]:
ds.shuffle()[0]["essay"]

'\nAs a Ship Engineer in the transportation industry, I recently tackled a complex issue involving the malfunctioning of a marine vessel\'s propulsion system. The inability to maintain consistent speed and power affected the vessel\'s efficiency and safety, which could lead to severe consequences if not addressed promptly. In this essay, I will explain how I used the Root Cause Analysis (RCA) strategy to solve this problem, its effectiveness and limitations, and my insights for future applicability.\n\nI chose RCA because it is a structured and disciplined approach to problem-solving that focuses on identifying the underlying reasons for an issue, rather than just addressing its symptoms. I considered other strategies such as Fault Tree Analysis (FTA) and Failure Mode and Effects Analysis (FMEA), but RCA was more suitable because it enabled me to identify the primary cause of the problem and eliminate it, rather than just listing possible failure modes or building a logical diagram to 

In [35]:
import re

def highlight_tokens_html(text, names, famous):
    """
    Generates HTML with names highlighted in yellow and famous names highlighted in green.
    """
    # Import the re module for regex operations

    # Prepare the HTML style
    style = "<style>.yellow { background-color: yellow; } .green { background-color: green; } .red { background-color: red; }</style>"

    # Initialize the HTML with the style tag
    highlighted_html = style + text

    # Function to escape special characters in names for regex
    def escape_names(names):
        return "|".join(re.escape(name) for name in names.split("|") if name)

    # Highlight famous names in green
    if famous:
        famous_pattern = r"\b(" + escape_names(famous) + r")\b"
        highlighted_html = re.sub(famous_pattern, r"<span class='green'>\1</span>", highlighted_html)

    # Highlight other names in yellow
    if names:
        names_pattern = r"\b(" + escape_names(names) + r")\b"
        highlighted_html = re.sub(names_pattern, r"<span class='yellow'>\1</span>", highlighted_html)

    # Highlight "URL" in red
    url_pattern = r"(URL)"
    highlighted_html = re.sub(url_pattern, r"<span class='red'>\1</span>", highlighted_html)

    # Replace newlines with <br> for HTML display
    highlighted_html = highlighted_html.replace("\n", "<br>")

    return highlighted_html

In [41]:
from IPython.display import HTML

name_key = "names"
famous_key = "famous_person"
text_key = "essay"

x = ds.shuffle()[0]

print("Names:", x[name_key])
print("Famous:", x[famous_key])

HTML(highlight_tokens_html(x[text_key], x[name_key], x[famous_key]))

Names: 
Famous: 


In [24]:
print(x["essay"])


Introduction

The complex problem I faced was staying relevant and innovative in the ever-evolving fashion industry. With constant fluctuations in trends, it's challenging to consistently create unique and captivating designs that cater to various styles and tastes. My success as a Fashion Accessory Designer hinges on my ability to tackle this problem effectively.

Methodology Selection

In addressing this intricate challenge, I chose a design thinking methodology. Design thinking is a human-centered approach that prioritizes empathy, experimentation, and iteration throughout the problem-solving process. I opted for this methodology due to its focus on understanding users' needs and preferences—an essential aspect of creating successful fashion accessories. Additionally, design thinking encourages creative solutions and adaptations for context, allowing me to draw inspiration from emerging trends and styles.

Application

I applied the design thinking methodology by first empathizing 

In [ ]:
<<Website URL>>
URL: <<Website URL>>
URL: <<website>>
URL: <<My website>>
<<URL to my website, where I discuss my work on utilizing data-driven approaches in financial analysis.>>
<

URL>

<
URL>
</s>

# Replace URLs to see if there are others of a different format

In [22]:
import re

# ds = ds.map(lambda x: {"modified": x["essay"].replace("<<URL>>", "<<POTATO>>")}, num_proc=4)
ds = ds.map(lambda x: {"modified": re.sub(r"<<.*>>", "<<POTATO>>", x["essay"])}, num_proc=4)

Map (num_proc=4):   0%|          | 0/3000 [00:00<?, ? examples/s]

In [23]:
len([x for x in ds["modified"] if "URL" in x])

145

In [24]:
temp = ds.filter(lambda x: "URL" in x["modified"])

Filter:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [32]:
from IPython.display import HTML

name_key = "names"
famous_key = "famous_person"
text_key = "essay"

x = temp.shuffle()[0]

print("Names:", x[name_key])
print("Famous:", x[famous_key])

HTML(highlight_tokens_html(x[text_key], x[name_key], x[famous_key]))

Names: Katalaya|Letha
Famous: Kerry Washington


In [33]:
print(x["essay"])


Introduction to the Problem

As a Telecommunications Line Installer and Repairer, I was confronted with a complex challenge when our team encountered a seemingly insurmountable issue during the installation of a new rural broadband network. The project involved laying over 100 miles of fiber-optic cables to connect remote communities to the internet. The complexity arose as we were working in a geographically challenging terrain, characterized by steep mountains and dense forests. Moreover, we had to ensure minimal environmental impact while adhering to strict project timelines and budgets. The successful completion of this project would significantly improve access to essential communication services and enhance the quality of life for the residents in these rural areas. The problem's complexity and significance necessitated a well-thought-out, strategic approach to overcome the multiple obstacles we faced.

Rationale Behind the Strategy Selection

After evaluating various approaches

In [91]:
# len([x for x in ds["modified"] if "URL" not in x and "www" in x])
len([x for x in ds["modified"] if "URL" not in x and ".com" in x])

22

In [94]:
[x for x in ds["modified"] if "URL" not in x and "www" in x][3]

'\nComplex Issue: The fashion industry has long been plagued by the issue of sustainability and ethical production. Fast fashion, which relies on cheap labor and materials, contributes significantly to pollution and waste. Additionally, the lack of transparency in supply chains often leaves consumers in the dark about the origins of their clothing. As a fashion agent, I wanted to tackle this issue and promote sustainable and ethical practices within the industry.\n\nStrategy Choice:\nTo address the problem of sustainability in the fashion industry, I chose to implement a strategy of collaboration and education. I decided to bring together like-minded industry professionals, including designers, buyers, and media, to promote sustainable and ethical fashion practices. This approach was chosen because it leverages my strengths in networking and relationship-building. Other potential strategies, such as advocating for policy change or launching a personal fashion line, were deemed less fea

In [72]:
def remove_duplicates(x):
    url_locations = []
    to_delete = []

    for match in re.finditer(".?URL.?", x):

        inner_matched = False
        for u in url_locations:
            if 0 < match.start() - u < 10:
                x = x[:match.start()] + x[match.end():]
                inner_matched = True
                break
        url_locations.append(match.start())
        
        if inner_matched:
            return remove_duplicates(x)
        
    return x


def remove_word_count(x):

    patterns = [
        "\nWord count:",
        "\nWord Count:",
        "\n(Word Count:",
        "\nEssay Length:"
    ]
    for p in patterns:
        if p in x and x.rindex(p) > 0.8*len(x):
            x = x[:x.index(p)]

    # (1092 words)

    return x

def remove_note(x):

    patterns = [
        "\nNote:",
        "\n(Note:",
       "\n[Note:",
       "\nPlease note",
    ]

    for p in patterns:
        if p in x and x.rindex(p) > 0.8*len(x):
            x = x[:x.index(p)]

    return x

def clean_URL(x):

    patterns = [
        r"\nURL: \[[^\n]+\]\n\n",
        r"\n\(URL: [^\n]+\n\n",
        r"\nPersonal URL: [^\n]+\n\n",
        r"\nPersonal URL: [^\n]+$",
    ]
    if "\nURL: [\n\n" in x:
        pass

    # pattern = 
    if "\n(URL:.*\n\n":
        pass
    if "\n[URL:.*\n\n":
        pass
    
    # pattern = 
    if "\nPersonal URL: [https://www.linkedin.com/in/[YourName]]":
        pass

def clean_essays(x):
    essay = x["essay"]
    names = x["names"]
    famous = x["famous_person"]

    # url_matches = re.finditer(r"\n[a-zA-Z\(\[ ]*URL[a-zA-Z\(\[\)\] :\.]*\n", essay)

    replaced = re.sub(r"\n[a-zA-Z\(\[ ]*URL[a-zA-Z\(\[\)\] :\./0-9'\"]*[\n\b]", " <URL> ", essay)

    url_locations = list(re.finditer("<URL>", replaced))
    other_urls = list(re.finditer("[URL]", replaced))

    hyperlink_pattern = r'(https?:\/\/(?:www\.)?[-a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,6}(?:[-a-zA-Z0-9@:%_\+.~#?&//=]*))'
    hyperlinks = list(re.finditer(hyperlink_pattern, replaced))

    other_hyperlinks = list(re.finditer(r'www\.[a-zA-Z0-9\-\.]+\.[a-z]{2,6}', replaced))

    to_delete = []

    for hyperlink in [*hyperlinks, *other_hyperlinks]:
        start = hyperlink.start()
        end = hyperlink.end()
        
        for url_location in url_locations:
            if 0 < start - url_location.end() < 5 or 0 < end - url_location.start() < 5:
                to_delete.append(hyperlink)
        
        for other_url in other_urls:
            if 0 < start - other_url.start() < 5 or 0 < end - other_url.end() < 5:
                to_delete.append(hyperlink)


    for hyperlink in to_delete:
        replaced = re.sub(re.escape(hyperlink.group(0)), " <URL> ", replaced)

    if "\nWord count:" in replaced:
        replaced = replaced[:replaced.index("\nWord count:")]
    if "\nWord Count:" in replaced:
        replaced = replaced[:replaced.index("\nWord Count:")]

    if "\nNote:" in replaced:
        replaced = replaced[:replaced.index("\nNote:")]

    replaced = remove_duplicates(replaced)


    return {
        "replaced": replaced
    }
    

# ds = ds.map(clean_essays)

In [362]:
x = all_ds.shuffle()[0]

        # #personalurl#
# [URL, remove brackets]
# [insert personal URL].
# URL removed
# [Insert Personal URL Here]

pattern =  "|".join([
        r"\nURL: [^\n]+\]\n\n", # URL: [https://www.linkedin.com/in/[YourName]]\n\n
        r"\nURL: \[[^\n]+$", # URL: [https://www.linkedin.com/in/[YourName]]
        r"\n[\(\[]URL: [^\n]+\n\n", # (URL: [https://www.linkedin.com/in/[YourName]]\n\n
        r"\nPersonal URL: [^\n]+\n\n", # Personal URL: https://www.linkedin.com/in/[YourName]\n\n
        r"\nPersonal URL: [^\n]+$", # Personal URL: https://www.linkedin.com/in/[YourName]
        r"\n ?\[URL\][\[\(][^\n]+\n\n", # [URL](http://www.example.com)\n\n
        r"\n\[URL\][\[\(][^\n]+$", # [URL](http://www.example.com)
        r"\n\[URL[^\n]+\]\n", # [URL: http://www.example.com]\n\n
        r"\n\[URL[^\n]+$", # [URL: http://www.example.com]
        r"\n\[URL\][^\n]+\n", # [URL: http://www.example.com]\n
    ])

count = 0
while len(re.findall(pattern, x["essay"])) == 0 and "URL" not in x["essay"]:
    x = all_ds.shuffle()[0]
    count += 1
    if count > 1000: break

essay = re.sub(pattern, " <URL> ", x["essay"])

print("Names:", x["names"])
print("Famous:", x["famous_person"])
print(re.findall(pattern, x["essay"]))

HTML(highlight_tokens_html(essay, x["names"], x["famous_person"]))

Names: Apollo
Famous: 
['\n[URL: https://www.systems-thinking.org/]']


In [314]:
re.findall(r"\n ?\[URL\][\[\(][^\n]+\n\n", x["essay"])

['\n [URL](https://www.telecom-experts.com/blog/case-study-telecom-equipment-specialist-tackles-network-challenge)\n\n']

In [372]:
[URL] www.linkedin.com/in/dedicated-paint-technician/

["\nThe Complex Problem of Brake Judder and Its Solution Through Applied Methodology\n\nAs an Automotive Brake Specialist, I have encountered numerous complex problems related to automotive brake systems. One prevalent issue that often challenges even the most experienced specialists is brake judder, which can compromise the safety and performance of vehicles. This essay will describe my application of a specific methodology to tackle this complex problem and critically evaluate its effectiveness.\n\nBrake judder manifests as a vibration or pulsating sensation when applying the brakes and is typically caused by unevenly worn brake discs or pads. The problem can worsen over time, leading to a significant decrease in braking performance and safety. Given its potential impact on both drivers and pedestrians, it is crucial to address brake judder effectively and efficiently.\n\nFor this complex problem, I chose to apply the Plan-Do-Check-Act (PDCA) methodology, which is a cyclical and iter

In [312]:
x["essay"]

"\nThe Challenge: A Multifaceted Telecom Network Dilemma\n\nAs a Telecommunications Equipment Specialist, I faced a complex challenge when a major client approached me with an issue. They were experiencing frequent network downtime, affecting their business operations and customer satisfaction. The client's telecom network spanned multiple buildings with a mix of old and new equipment, making the issue difficult to diagnose and resolve.\n\nSelecting the Right Tool: Analytics and Expertise\n\nI decided to deploy network analytics software as my primary tool, supplemented by my expertise in telecom systems and problem-solving skills. Analytics could quickly process vast amounts of data and reveal patterns, trends, and potential causes of the downtime. Moreover, the software would help me efficiently monitor and manage the network, maximizing uptime and minimizing service disruptions.\n\nChoosing this tool over alternative options, like manual diagnostics or reactive maintenance, was a st

In [364]:
m

[<re.Match object; span=(0, 5), match='[URL]'>]

In [269]:
hyperlink_pattern = r'(https?:\/\/)?(?:www\.)?[-a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b(?:[-a-zA-Z0-9@:%_\+.~#?&//=]*)'

hyperlinks = list(re.finditer(hyperlink_pattern, x["essay"]))

In [270]:
len(hyperlinks)

2

In [305]:
hyperlinks

[<re.Match object; span=(5659, 5716), match='www.financialanalystexpert.com/blog/stock-splits->,
 <re.Match object; span=(5718, 5782), match='http://www.financialanalystexpert.com/blog/stock->]

In [420]:
x["essay"]

'\nIntroduction:\n\nAs an electrical assembler in the manufacturing industry, I encounter a myriad of complex issues on a daily basis. From rewiring malfunctioning systems to interpreting complex engineering blueprints, my role demands a high level of problem-solving acumen. However, one issue stands out as particularly challenging and significant: the optimization of electrical assembly lines to increase efficiency and productivity while maintaining quality control.\n\nStrategy Choice:\n\nTo tackle this complex issue, I chose a strategy known as the DMAIC (Define, Measure, Analyze, Improve, Control) approach. This strategy is commonly used in Six Sigma methodology and is effective in solving complex problems by breaking them down into manageable phases. I chose this strategy because of its comprehensive and analytical nature, as well as its emphasis on data-driven decision-making. Alternative strategies such as the PDCA (Plan, Do, Check, Act) cycle and the 8D problem-solving process w

In [ ]:
from pathlib import Path
from datasets import concatenate_datasets, Dataset


all_ds = concatenate_datasets([Dataset.from_parquet(str(x)) for x in Path("../data/essays").glob("mixtral-*.pq")])

In [4]:
all_ds

Dataset({
    features: ['id', 'essay', 'prompt', 'names', 'famous_person', 'bio'],
    num_rows: 6200
})

In [6]:
len(set(all_ds["id"])), len([x for x in all_ds["essay"] if "URL" in x]), len([x for x in all_ds["essay"] if "[URL]" in x])

(6200, 5761, 4194)

In [9]:
import re

def isolate_url(essay):
    chunks = essay.split("\n")

    final_chunks = []

    current = ""

    if "URL" in chunks[0]:
        current += chunks[0]
        
    for i in range(1, len(chunks) - 1):
        if "URL" in chunks[i]:
            if len(current) == 0:
                current += 

    return [x for x in chunks if "URL" in x]

temp = all_ds.map(lambda x: {"url": isolate_url(x["essay"])}, num_proc=4)


Map (num_proc=4):   0%|          | 0/6200 [00:00<?, ? examples/s]

In [10]:
temp = temp.filter(lambda x: len(x["url"]) > 0, num_proc=4)

Filter (num_proc=4):   0%|          | 0/6200 [00:00<?, ? examples/s]

In [14]:
temp.shuffle()["url"]

[['[URL]'],
 ['[URL: www.mypersonalwebsite.com/leansixsigma]'],
 ['[URL]'],
 ['[URL]'],
 ['For more information about my work in this area, please visit my personal URL: [URL]'],
 ['[URL]'],
 ['[URL: www.myurologist.com]'],
 ['[URL: LinkedIn profile]'],
 ['US EPA. (2021). Risk Communication. Retrieved from <https://www.epa.gov/risk/risk-communication> [URL]'],
 ['[URL] Relevant personal project or portfolio page showcasing the robot and its capabilities.'],
 ['[URL] for more about retail store planning, visit: [www.retailstoreplanning.com](http://www.retailstoreplanning.com)'],
 ['Savery, J. R. (2006). Overview of problem-based learning: Definitions and distinctions. Interdisciplinary Journal of Problem-Based Learning, 1(1), 9-20. [URL]',
  '[URL] = <https://www.medicaleducationonline.org/site/misc/Overview_of_PBL.pdf>'],
 ['[URL: Example of Retail Loss Prevention Strategies](https://www.losspreventionmedia.com/articles/view/3260)'],
 ['[URL] www.powerlineinstallationandrepair.com/sixs

In [73]:
all_ds = all_ds.map(lambda x: {"no_note": remove_note(x["essay"])}, num_proc=4)

Map (num_proc=4):   0%|          | 0/6200 [00:00<?, ? examples/s]

In [74]:
[x for x in all_ds["no_note"] if "note" in x.lower() and x.lower().rindex("note")/len(x) > 0.5*len(x)]

[]

In [66]:
all_ds = all_ds.map(lambda x: {"no_word_count": remove_word_count(x["no_note"])}, num_proc=4)

Map (num_proc=4):   0%|          | 0/6200 [00:00<?, ? examples/s]

In [67]:
[x for x in all_ds["no_word_count"] if "word count" in x.lower() and x.lower().rindex("word count")/len(x) > 0.5*len(x)]

[]

In [80]:
len([x for x in all_ds["essay"] if "essay length" in x.lower()])

10

In [84]:
[x[x.lower().index("essay length")-10:] for x in all_ds["essay"] if "essay length" in x.lower()]

['y [/URL]\n\nEssay Length: 507 words',
 '.\n\n[URL]\n\nEssay Length: 938 words',
 ' change.\n\nEssay Length: 600 words (Please note that the word count is intentionally kept below the specified range to accommodate potential platform-specific character limits.)\n[URL]: <https://www.interaction-design.org/literature/topics/design-thinking>',
 'to come.\n\nEssay Length: 829 words\n\nLink to personal URL: [www.linkedin.com/in/nonprofitadvocate](http://www.linkedin.com/in/nonprofitadvocate)',
 'e: [URL]\n\nEssay Length: 723 words.',
 'ite.com]\n\nEssay Length: 700 words.',
 "he actual essay length should be adjusted to reach the specified range (1000-1500 words) by expanding on the given points and providing additional context and examples where necessary. Additionally, the reference to the personal URL should be replaced with a real link when the essay is finalized. The text here serves as a model and should be customized to accurately reflect the writer's personal experiences and insigh

In [91]:
len([x for x in all_ds["essay"] if "---" in x.lower()])

94

In [92]:
[x for x in all_ds["essay"] if "---" in x][:10]

['\nThe Scene of the Struggle\n------------------------\n\nAs an Avionics Technician in the transportation industry, I often encounter intricate and challenging problems that require careful analysis and resolution. However, none have been as daunting as the one I faced a few months ago. I was working on a Boeing 737 MAX aircraft, which had just undergone significant modifications to its avionic systems following the grounding of the entire fleet. The problem arose when the aircraft\'s autopilot system began displaying erratic behavior during test flights. The autopilot would intermittently disengage without any warning or explanation, posing a significant safety risk for both passengers and crew.\n\nThe complexity of this challenge was amplified by several factors, including the aircraft\'s advanced avionic systems, the intricate interplay between multiple software applications, and the high stakes involved in ensuring passenger safety. Furthermore, the grounding of the 737 MAX had al

In [93]:
len([x for x in all_ds["essay"] if "[/end]" in x])

8

In [95]:
len([x for x in all_ds["essay"] if "<!" in x])

1

In [90]:
from IPython.display import HTML

x = all_ds.shuffle()[0]

# while x["essay"] == x["no_note"]:
#     x = all_ds.shuffle()[0]

print("Names:", x["names"])
print("Famous:", x["famous_person"])

# print(x["essay"])


HTML(highlight_tokens_html(x["no_note"], x["names"], x["famous_person"]))

Names: Omid|Lynda
Famous: 


In [431]:
essays = {i: (e, n, f) for i, e, n, f in zip(all_ds["id"], all_ds["essay"], all_ds["names"], all_ds["famous_person"]) if "URL" in e}

In [434]:
good_ids = set()
bad_ids = set()

In [436]:
bad_ids

set()

In [437]:
good_ids

{'kbbrnhmk'}

In [439]:
import gradio as gr
import datetime
import pickle



def bad_click(id):
    bad_ids.add(id)

    with open("bad_ids.pkl", "wb") as f:
        pickle.dump(bad_ids, f)

    return next(x for x in essays.keys() if x not in good_ids and x not in bad_ids)

def good_click(id):

    good_ids.add(id)

    with open("good_ids.pkl", "wb") as f:
        pickle.dump(good_ids, f)

    return next(x for x in essays.keys() if x not in good_ids and x not in bad_ids)


def show_html(id):
    e, n, f = essays[id]
    return highlight_tokens_html(e, n, f)

first_id = next(x for x in essays.keys() if x not in good_ids and x not in bad_ids)

with gr.Blocks() as demo:
    with gr.Column():
        with gr.Row():
            bad = gr.Button("Bad")
            good = gr.Button("Good")
        id = gr.Textbox(value=first_id)
        html = gr.HTML(value=show_html(first_id))

    id.change(fn=show_html, inputs=[id], outputs=[html])
    bad.click(fn=bad_click, inputs=[id], outputs=[id])
    good.click(fn=good_click, inputs=[id], outputs=[id])


demo.launch()

Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.
